## Multiple Chain w/ LCEL

Runnables can be used to chain multiple Chains, use RunnablePassthrough to pass the output to multiple downstream chains simultaneously.

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

In this example, use Runnable to acheive discussion on the same topic among multiple AIs：

```
    Input topic
       |
       |
original viewpoint
      / |\
     /  |  \
supporting| opposing
     \  | /
      \ |/
     conclusion
```

In [1]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

planner = (
    ChatPromptTemplate.from_template("Generate arguments on the following topic: {input}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# Create a processing chain for supporting arguments, listing the positive or favorable aspects of the basic response.
arguments_for = (
    ChatPromptTemplate.from_template(
        "List the positive or favorable aspects of {base_response}"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# Create a processing chain for counterarguments, listing the negative or unfavorable aspects of the basic response.
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the negative or unfavorable aspects of {base_response}"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

#Create a final responder that synthesizes the original response along with the supporting and opposing arguments to generate a final conclusion.
final_responder = (
    ChatPromptTemplate.from_messages( # Build a multi-message chat prompt
        [
            ("ai", "{original_response}"),
            ("human", "Supporting arguments:\n{results_1}\n\nOpposing arguments:\n{results_2}"),
            ("system", "Generate a final response after providing critiques"),
        ]
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# Build a complete processing chain, from generating arguments to listing pros and cons, and finally producing the final response.
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),  # Take the value from the key base_response in the planner's output, and pass it to the key original_response expected by final_responder
    }
    | final_responder
)

In [2]:
print(chain.invoke({"input": "AI in 2025"}))

### Final Response

The exploration of AI's potential in 2025 reveals a dual-edged sword, showcasing both promising advancements and significant challenges that merit careful consideration. 

On the positive side, the arguments presented highlight a future where AI significantly enhances productivity, revolutionizes healthcare, creates smarter urban environments, personalizes education, and facilitates groundbreaking research. These developments could lead to substantial societal advancements, improved living standards, and global collaboration. Moreover, the potential for AI to contribute to sustainability and support vulnerable populations underscores its transformative potential.

However, a thorough critique of the opposing arguments illuminates serious concerns that cannot be overlooked. The risk of job displacement and economic inequality raises pressing questions about workforce readiness and societal equity. Ethical dilemmas associated with AI decision-making, including account

#### Streaming the output

In [4]:
## The final output of the chain is processed by StrOutputParser, so it can be directly streamed as output s
for s in chain.stream({"input": "World Economy in 2025"}):
    print(s, end="", flush=True)

The arguments put forth for the positive outlook of the world economy in 2025 are compelling, reflecting a potential landscape characterized by technological advancements and growth in emerging markets. However, it's important to critically analyze these claims in the context of the challenges that may undermine this optimistic scenario.

### Critiques of the Positive Outlook Arguments

1. **Technological Advancements**: While technology can enhance productivity, the benefits may not be evenly distributed. Automation can lead to job displacement, particularly in low-skill sectors, potentially exacerbating unemployment and inequality.

2. **Sustainability and Green Economy**: Although the green economy has growth potential, there may be resistance due to vested interests in fossil fuels and traditional industries. Transitioning to sustainable practices can also be costly and complex.

3. **E-commerce and Digital Transformation**: The digital divide presents a significant challenge. Not 